<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Full aircraft sizing with Powertain Builder

The Powertrain Builder is a newly developed toolset that enables flexible exploration of powertrain architectures. It can be integrated into FAST-OAD_CS23 missions. This notebook uses the Pipistrel Velis Electro as a case study to demonstrate its functionality.

## 1. Setting up and initializing the problem

In [ ]:
import os.path as pth
import logging
import shutil
import fastoad.api as oad
import fastga_he.api as oad_he

# Define relative path
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"

# Define file paths
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "pipistrel_configuration.yml")
PT_FILE = pth.join(WORK_FOLDER_PATH, "pipistrel_assembly.yml")
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "pipistrel_source.xml")

# For having log messages on screen
logging.basicConfig(level=logging.WARNING, format="%(levelname)-8s: %(message)s")

In [ ]:
# copy all the useful YML files into the workdir directory

shutil.copy(pth.join(DATA_FOLDER_PATH, "pipistrel_configuration.yml"), CONFIGURATION_FILE)
shutil.copy(pth.join(DATA_FOLDER_PATH, "pipistrel_assembly.yml"), PT_FILE)

The following is the propulsion system diagram of the Pilot Operation Handbook (POH) for the Pipistrel Velis Electro. The backbone of the Pipistrel Velis Electro powertrain is formed by the components **<span style="color:orange">highlighted in orange</span>**. Those are the components that will be added in the powertrain file which is how the archtecture of the propulsive systems is specified to FAST-OAD-CS23-HE.

<img src="./img/highlight_pipistrel.gif">

With all the components identified, let's make sure that the powertrain described in the powertrain configuration file corresponds to the diagram above. To do so, FAST-OAD-GA-HE offers a visualization function of the powertrain as a network. Its use is displayed in the following cell. 

In [ ]:
from IPython.display import IFrame

NETWORK_FILE = pth.join(WORK_FOLDER_PATH, "network_pipistrel.html")

oad_he.power_train_network_viewer(power_train_file_path=PT_FILE, network_file_path=NETWORK_FILE)

IFrame(src=NETWORK_FILE, width="100%", height="500px")

For a better architecture visualization run the following cell, it will open the HTML file in a separate tab.

In [ ]:
import webbrowser

webbrowser.open_new_tab(pth.join(DATA_FOLDER_PATH, "network_pipistrel.html"))

With the powertrain configuration verified with the architecture from the Pipistrel Velis Electro's pilot operational handbook, let's move on to the generation of inputs.

In [ ]:
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

## 2. Running Pipistrel Velis Electro's MDA 

Now we run the problem. As the performances computation was made to stall when not enough progress is made or when it has already converged and can't do more, we will catch those warnings for a cleaner display.

In [ ]:
import warnings
import openmdao.api as om

warnings.filterwarnings("ignore", category=om.OpenMDAOWarning)

configurator = oad.FASTOADProblemConfigurator(CONFIGURATION_FILE)
problem = configurator.get_problem(read_inputs=True)
problem.setup()
problem.set_val("data:weight:aircraft:MTOW", units="kg", val=600.0)
problem.run_model()
problem.write_outputs()

print(problem.get_val("data:weight:aircraft:MTOW", units="kg"))

## 3. Post-processing

After obtaining the result, we can now check the powertrain perormances with the data saved during sizing mission computation using the ``PerformancesViewer``.

In [ ]:
MISSION_DATA_FILE = pth.join(WORK_FOLDER_PATH, "pipistrel_mission_data.csv")
PT_DATA_FILE = pth.join(WORK_FOLDER_PATH, "pipistrel_power_train_data.csv")

perfo_viewer = oad_he.PerformancesViewer(
    power_train_data_file_path=PT_DATA_FILE,
    mission_data_file_path=MISSION_DATA_FILE,
    plot_height=800,
)

# Uncomment next lines if you want raw data
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 200)
# print(perfo_viewer.data)

For the mass breakdown of the powertrain components, you can use the ``power_train_mass_breakdown``.

In [ ]:
fig = oad_he.power_train_mass_breakdown(
    problem.output_file_path,
    power_train_file_path=PT_FILE,
)
fig.update_layout(height=800)

To further investigate the results of the run, use the ``variable_viewer``.

In [ ]:
oad.variable_viewer(problem.output_file_path)

Use ``write_n2`` to observe the problem N2 diagram structure in another window.

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
webbrowser.open_new_tab(N2_FILE)

## 4. Propeller efficiency optimization with OpenMDAO problem definition

This part use ``PerformancesPropeller`` to demonstrate how to run optimization problem for specific subsystems in the FAST-OAD-CS23-HE framework with an OpenMDAO approach.

In [ ]:
import openmdao.api as om
from stdatm import Atmosphere
from fastga_he.models.propulsion.components.propulsor.propeller import PerformancesPropeller

# Define problem in OpenMDAO
problem = om.Problem(reports=False)
model = problem.model

# Add subsystems into the problem
model.add_subsystem(
    "ivc_rpm",
    om.IndepVarComp(
        "data:propulsion:he_power_train:propeller:propeller_1:rpm_mission", 2300, units="1/min"
    ),
    promotes=["*"],
)
model.add_subsystem(
    "propeller_perf",
    PerformancesPropeller(propeller_id="propeller_1", number_of_points=1),
    promotes=["*"],
)

# Define optimization driver and optimizer options
problem.driver = om.ScipyOptimizeDriver()
problem.driver.options["optimizer"] = "differential_evolution"
problem.driver.options["maxiter"] = 1000
problem.driver.options["tol"] = 1e-4

# DEfine the design variables range of the problem
problem.model.add_design_var(
    "data:propulsion:he_power_train:propeller:propeller_1:solidity", lower=0.1, upper=0.4
)
problem.model.add_design_var(
    "data:propulsion:he_power_train:propeller:propeller_1:activity_factor", lower=50, upper=250
)
problem.model.add_design_var(
    "data:propulsion:he_power_train:propeller:propeller_1:blade_twist",
    lower=10,
    upper=35,
    units="deg",
)
problem.model.add_design_var(
    "data:propulsion:he_power_train:propeller:propeller_1:diameter",
    lower=40,
    upper=80,
    units="inch",
)

# Define optimization objective function
problem.model.add_objective("efficiency", scaler=-1.0)

# Define problem driver and problem options
problem.nonlinear_solver = om.NewtonSolver(solve_subsystems=True)
problem.nonlinear_solver.options["iprint"] = 0
problem.nonlinear_solver.options["maxiter"] = 100
problem.nonlinear_solver.options["rtol"] = 1e-4
problem.linear_solver = om.DirectSolver()

problem.model.approx_totals()

# Problem setup
problem.setup()

# Set up problem initial values ( Original output from sizing)
problem.set_val("altitude", val=610.0, units="m")
problem.set_val("density", val=Atmosphere(610.0).density, units="kg/m**3")
problem.set_val("thrust", val=536.5, units="N")
problem.set_val("true_airspeed", val=47.8, units="knot")
problem.set_val("data:propulsion:he_power_train:propeller:propeller_1:solidity", 0.135)
problem.set_val("data:propulsion:he_power_train:propeller:propeller_1:activity_factor", 125.0)
problem.set_val(
    "data:propulsion:he_power_train:propeller:propeller_1:blade_twist", val=20.0, units="deg"
)
problem.set_val(
    "data:propulsion:he_power_train:propeller:propeller_1:diameter", val=164.0, units="cm"
)

problem.set_solver_print(level=0)

# Run problem
problem.run_driver()

# output optimization results
print(
    f"Propeller solidity: {problem.get_val('data:propulsion:he_power_train:propeller:propeller_1:solidity')}"
)
print(
    f"Propeller activity factor: {problem.get_val('data:propulsion:he_power_train:propeller:propeller_1:activity_factor')}"
)
print(
    f"Propeller blade twist (deg): {problem.get_val('data:propulsion:he_power_train:propeller:propeller_1:blade_twist', units='deg')}"
)
print(
    f"Propeller diameter (inch): {problem.get_val('data:propulsion:he_power_train:propeller:propeller_1:diameter', units='inch')}"
)
print(f"Propeller efficiency: {problem.get_val('efficiency')}")